### Python Code by Lloyd A. Courtenay. Last Updated 10/02/2021
### Python Code for the training of NSVMs

Note for NSVM Tensorflow V.2.3. or higher is required

In [10]:
import os
os.chdir("") # Set Directory

# load libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.constraints import unit_norm
from tensorflow.keras.callbacks import *
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers.experimental import RandomFourierFeatures
import pandas as pd
import numpy as np
import math
from numpy.random import randint, rand, randn, random, choice
from numpy import ones, zeros, vstack
import sklearn
from sklearn.preprocessing import LabelEncoder
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error
from sklearn.svm import *
import hyperopt
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin

# Print package versions and ensure they have loaded correctly
print("Versions:")
print(f"Tensorflow {tf.__version__}")
print(f"Keras {keras.__version__}")
print(f"Numpy {np.__version__}")
print(f"Pandas {pd.__version__}")
print(f"SciKit Learn {sklearn.__version__}")
print(f"HyperOpt: {hyperopt.__version__}")

# Set random number generator
seed_value = 100
np.random.seed(seed_value)

Versions:
Tensorflow 2.3.1
Keras 2.4.0
Numpy 1.17.3
Pandas 0.25.2
SciKit Learn 0.22.1
HyperOpt: 0.2.3


# Cyclic Learning Rate

In [2]:
class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

# Swish Activation Function

In [3]:
def swish(x, beta = 1):
    return(x * K.sigmoid(beta * x))
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation
get_custom_objects().update({"swish":Activation(swish)})

# Load Data
- Ensure that the file with PC Scores for training are in the directory and labelled "Data_For_Train.csv", using a commas for column delimiters.
- Ensure that the file with PC Scores for testing are in the directory and labelled "Data_For_Test.csv", using a commas for column delimiters.

In [4]:
# Load data
dataframe = pd.read_csv("Data_for_Train.csv", delimiter = ",", header = None)
dataframe = dataframe.drop(dataframe.index[0])
X = dataframe.values[:,1:6].astype(float)
Y = dataframe.iloc[:,0].astype("category")
test = pd.read_csv("Data_for_Test.csv", delimiter = ",", header = None)

test = test.drop(test.index[0])
X_test = test.values[:,1:6].astype(float)
Y_test = test.iloc[:,0].astype("category")

encoder = LabelEncoder()
encoder.fit(Y)
Y_encoded = encoder.transform(Y)
dummy_Y =  tf.keras.utils.to_categorical(Y_encoded)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y,
                                                  test_size = 0.2,
                                                  random_state = 666)

train_encoded_Y = encoder.transform(Y_train)
val_encoded_Y = encoder.transform(Y_val)
dummy_Y_train = tf.keras.utils.to_categorical(train_encoded_Y) # One Hot Encoded - Convert Integers to Dummy 
dummy_Y_val = tf.keras.utils.to_categorical(val_encoded_Y)

encoded_Y_test = encoder.transform(Y_test)
dummy_Y_test = tf.keras.utils.to_categorical(encoded_Y_test)

key = pd.DataFrame(index = range(Y.cat.categories.size), 
                   columns = ["key_value", "sample"])
for i in range(Y.cat.categories.size):
    key.loc[i, "key_value"] = i
    key.loc[i, "sample"] = Y.cat.categories[i]
key

,key_value,sample
0,0,Hyena
1,1,Leopard
2,2,Licaon
3,3,Lion


# Define NN Model

In [5]:
visible = Input(shape = (5,))

transform = RandomFourierFeatures(output_dim = 500,
                                  kernel_initializer = "laplacian",
                                  trainable = True,
                                  scale = 0.26) (visible)
batch_norm = BatchNormalization() (transform)
dropout = Dropout(0.5) (batch_norm)
hidden = Dense(250,
               activation = "swish",
               kernel_initializer = "random_normal",
               kernel_regularizer = l2(l = 0.0001)) (dropout)
batch_norm = BatchNormalization() (hidden)
dropout = Dropout(0.5) (batch_norm)
hidden = Dense(250,
               activation = "swish",
               kernel_initializer = "random_normal",
               kernel_regularizer = l2(l = 0.0001)) (dropout)
batch_norm = BatchNormalization() (hidden)
dropout = Dropout(0.2) (batch_norm)
hidden = Dense(key.shape[0], activation = "softmax") (dropout)
model = Model(inputs = visible, outputs = hidden)

clr = CyclicLR(base_lr = 0.0001,
              max_lr = 0.01,
              step_size = 16, # maybe try 8
              mode = "triangular2")

model.compile(loss='categorical_crossentropy',
              optimizer="adam", metrics=['categorical_accuracy'])

history = model.fit(X, dummy_Y,
                    epochs = 1000,
                    batch_size = 32,
                    validation_split = 0.8,
                    callbacks = [clr],
                    verbose = 0
                    )

# Neural SVM

In [6]:
# Extract Trained Weights excluding the final layer

weights = model.get_weights()
NSVM_weights = weights[:19]

# create model excluding the final layer

visible = Input(shape = (5,))
transform = RandomFourierFeatures(output_dim = 500,
                                  kernel_initializer = "laplacian",
                                  trainable = True,
                                  scale = 0.26) (visible)
batch_norm = BatchNormalization() (transform)
dropout = Dropout(0.5) (batch_norm)
hidden = Dense(250,
               activation = "swish",
               kernel_initializer = "random_normal",
               kernel_regularizer = l2(l = 0.0001)) (dropout)
batch_norm = BatchNormalization() (hidden)
dropout = Dropout(0.5) (batch_norm)
hidden = Dense(250,
               activation = "swish",
               kernel_initializer = "random_normal",
               kernel_regularizer = l2(l = 0.0001)) (dropout)
batch_norm = BatchNormalization() (hidden)

# redefine model without the final activation

neural_model = Model(inputs = visible, outputs = batch_norm)

# Restore weights

neural_model.set_weights(NSVM_weights)

### Save Trained Weights

In [ ]:
neural_model.save("Neural_Model.h5")

# to load the weights you have to redefine the model and type neural_model.load_weights("Model.h5")

### Use trained neural network for feature extraction

In [7]:
X_neural_train = neural_model.predict(X)
X_neural_test = neural_model.predict(X_test)

## Use extracted features to tune a Linear SVM activation layer
### Bayesian Hyperparameter optimization of SVM

In [8]:
space = {"C" : hp.uniform("C", 0, 1000)}
def objective(space):
    neural_svm = SVC(kernel = "linear",
                     C = space["C"]
                    )
    accuracy = cross_val_score(neural_svm, X_neural_train,
                               Y_encoded, cv = 10).mean()
    return {"loss": -accuracy, "status": STATUS_OK }
trials = Trials()
best = fmin(fn = objective,
            space = space,
            algo = tpe.suggest,
            max_evals = 100,
            trials = trials
           )

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:52<00:00,  1.12s/trial, best loss: -0.9460000000000001]


### Final Training of SVM Activation Layer

In [9]:
NSVM = SVC(kernel = "linear",
           C = best["C"],
           probability = True,
          ).fit(X_neural_train, Y_encoded)

prediction_NSVM = NSVM.predict(X_neural_test)
accuracy = classification_report(encoded_Y_test, prediction_NSVM)

print(accuracy)
print(confusion_matrix(encoded_Y_test, prediction_NSVM))

              precision    recall  f1-score   support

           0       0.93      0.98      0.95        85
           1       0.93      0.94      0.93        84
           2       0.98      0.91      0.94        87
           3       0.95      0.96      0.95        77

    accuracy                           0.95       333
   macro avg       0.95      0.95      0.95       333
weighted avg       0.95      0.95      0.95       333

[[83  1  0  1]
 [ 2 79  2  1]
 [ 1  5 79  2]
 [ 3  0  0 74]]
